In [4]:
import os
os.environ["OMP_NUM_THREADS"] = "16"
os.environ["MKL_NUM_THREADS"] = "16"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"
from threadpoolctl import threadpool_limits, threadpool_info

for lib in threadpool_info():
    print(lib['internal_api'], lib.get('num_threads'), lib.get('prefix', ''))

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import datacompy
import os, sys
import numpy as np
import re
import ast
from pathlib import Path
import openpyxl
import itertools
import torch

# narzedzia
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    cross_val_predict,
    learning_curve,
    RepeatedStratifiedKFold,
    GridSearchCV,
    RandomizedSearchCV
)
from collections import Counter
from sklearn.metrics import classification_report, confusion_matrix, silhouette_score, accuracy_score, pairwise_distances, make_scorer, precision_score, f1_score, roc_auc_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import tree
from sklearn.tree import plot_tree
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from scipy import stats
from scipy.stats import chi2_contingency, f_oneway, friedmanchisquare, wilcoxon
from scipy.spatial import distance
from joblib import dump, load

# modele
from xgboost import XGBClassifier, XGBRFClassifier
#from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# methods
from imblearn.under_sampling import ClusterCentroids, NearMiss
from scipy.optimize import differential_evolution
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN, HDBSCAN
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor, KNeighborsClassifier, NearestNeighbors
from sklearn.inspection import permutation_importance
from scipy.spatial.distance import euclidean
from imblearn.over_sampling import SMOTE, BorderlineSMOTE
from ctgan import CTGAN

from threadpoolctl import threadpool_limits, threadpool_info


Please note that you are missing the optional dependency: fugue. If you need to use this functionality it must be installed.
Please note that you are missing the optional dependency: snowflake. If you need to use this functionality it must be installed.
Please note that you are missing the optional dependency: spark. If you need to use this functionality it must be installed.
Python 3.12 and above currently is not supported by Spark and Ray. Please note that some functionality will not work and currently is not supported.


### Read Datasets

In [ ]:
brddata1 = pd.read_csv("D:\\ml\\undersampling_data\\data\\unsw\\oversampling\\borderline_data.csv")
brddata3 = pd.read_csv("D:\\ml\\undersampling_data\\data\\unsw\\oversampling\\borderline3_data.csv")
GANdata = pd.read_csv("D:\\ml\\undersampling_data\\data\\unsw\\oversampling\\GAN_data.csv")
GANdata3 = pd.read_csv("D:\\ml\\undersampling_data\\data\\unsw\\oversampling\\GAN3_data.csv")
smotedata1 = pd.read_csv("D:\\ml\\undersampling_data\\data\\unsw\\oversampling\\smote_data.csv")
smotedata3 = pd.read_csv("D:\\ml\\undersampling_data\\data\\unsw\\oversampling\\smote3_data.csv")

X_train = pd.read_csv("D:\\ml\\undersampling_data\\data\\unsw\\X_train.csv")
y_train = pd.read_csv("D:\\ml\\undersampling_data\\data\\unsw\\y_train.csv")

KeyboardInterrupt: 

In [ ]:
mix_df = pd.concat([brddata1, brddata3, GANdata, GANdata3, smotedata1, smotedata3], ignore_index=True)
smote_df = pd.concat([smotedata1, smotedata3], ignore_index=True)
borderline_df = pd.concat([brddata1, brddata3], ignore_index=True)
GAN_df = pd.concat([GANdata, GANdata3], ignore_index=True)
original_df = pd.concat([X_train, y_train], axis=1)

mix_df.to_csv("D:\\ml\\undersampling_data\\data\\unsw\\syntheticdf\\mix_data.csv", index=False)
smote_df.to_csv("D:\\ml\\undersampling_data\\data\\unsw\\syntheticdf\\smote_data.csv", index=False)
borderline_df.to_csv("D:\\ml\\undersampling_data\\data\\unsw\\syntheticdf\\borderline_data.csv", index=False)
GAN_df.to_csv("D:\\ml\\undersampling_data\\data\\unsw\\syntheticdf\\GAN_data.csv", index=False)
original_df.to_csv("D:\\ml\\undersampling_data\\data\\unsw\\syntheticdf\\original_data.csv", index=False)

In [2]:
#oversampling data
original_df = pd.read_csv("D:\\ml\\undersampling_data\\data\\unsw\\syntheticdf\\original_data.csv")
#mix_df = pd.read_csv("D:\\ml\\undersampling_data\\data\\unsw\\syntheticdf\\mix_data.csv")
smote_df = pd.read_csv("D:\\ml\\undersampling_data\\data\\unsw\\syntheticdf\\smote_data.csv") 
#gan_df = pd.read_csv("D:\\ml\\undersampling_data\\data\\unsw\\syntheticdf\\gan_data.csv")
borderline_df = pd.read_csv("D:\\ml\\undersampling_data\\data\\unsw\\syntheticdf\\borderline_data.csv")

X_test = pd.read_csv("D:\\ml\\undersampling_data\\data\\unsw\\X_test.csv")

#Before undersampling
print(original_df["Label"].value_counts())
count1=original_df["Label"].value_counts().sum()
count2=abs((original_df['Label']==0).sum() - (original_df['Label']==1).sum())
print(f"Before undersampling: {count2}")
print(f"After number of samples: {count1+count2}")

Label
0    1370400
1      61898
Name: count, dtype: int64
Before undersampling: 1308502
After number of samples: 2740800


In [ ]:
gan_df["Label"].value_counts()

Label
1    3925506
Name: count, dtype: int64

In [3]:
#Sum up all df 
#sum_all_data = pd.concat([smote_df, gan_df, borderline_df, original_df], axis=0, ignore_index=True)

#Split data
#X_mix, y_mix = mix_df.drop(columns=["Label", "source"]), mix_df["Label"]
X_smote, y_smote = smote_df.drop(columns=["Label", "source"]), smote_df["Label"]
#X_GAN, y_GAN = gan_df.drop(columns=["Label", "source"]), gan_df["Label"]
X_borderline, y_borderline = borderline_df.drop(columns=["Label", "source"]), borderline_df["Label"]

#Dictionary
data = {}
#data["mix"] = (X_mix, y_mix)
data["smote"] = (X_smote, y_smote)
#data["GAN"] = (X_GAN, y_GAN)
data["borderline"] = (X_borderline, y_borderline)

compare = {}
#compare["mix"] = mix_df
compare["smote"] = smote_df
#compare["GAN"] = gan_df
compare["borderline"] = borderline_df

### Undersampling

In [4]:
base = Path("D:/ml/undersampling_data/data/unsw/reduced")
base2 = Path("D:/ml/undersampling_data/models/kmeans&centroids2")

df_ = {}

for name in data.keys() & compare.keys():
    # Read data from dictionary
    X_train, y_train = data[name]
    compare_df = compare[name]
    
    # Select majority class
    X_majority = X_train[y_train == 1]
    
    KM = KMeans(n_clusters=(int)(count2))
    with threadpool_limits(limits=16):
    # Apply KMeans clustering
        kmeans = KM.fit(X_majority)
    
    # Create a DataFrame for centroids
    X_majority_reduced = pd.DataFrame(kmeans.cluster_centers_, columns=X_train.columns)
    y_majority_reduced = pd.Series([1] * (int)(count2), name="Label") 
    
    # Combine reduced majority class with original minority class
    df_majority = pd.concat([X_majority_reduced, y_majority_reduced], axis=1).reset_index(drop=True)
    
    # Add source column if not present
    df_majority["source"] = None
    missing_source = df_majority[df_majority["source"].isna()]
    if not missing_source.empty:
        df_majority.loc[df_majority["source"].isna(), "source"] = "centroid" 
    
    print(df_majority)
    df_majority = df_majority.reindex(columns=original_df.columns, fill_value=0.0)
    
    # Combine with original minority class
    df_[name] = pd.concat([df_majority, original_df], axis=0).reset_index(drop=True)  
    
    print(df_[name].info())
    
    # Save to CSV if file does not exist
    file_path = base / f"{name}_KM_centroids.csv"

    if os.path.exists(file_path):
        print("File exists.")
    else:
        df_[name].to_csv(file_path, index=False)
        print("File saved.")

c:\Users\Mati\AppData\Local\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] Nie można odnaleźć określonego pliku
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\Mati\AppData\Local\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
        "wmic CPU Get NumberOfCores /Format:csv".split(),
        capture_output=True,
        text=True,
    )
  File "c:\Users\Mati\AppData\Local\anaconda3\Lib\subprocess.py", line 554, in run
    with Popen(*popenargs, **kwargs) as process:
         ~~~~~^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Mati\AppData\Local\anaconda3\Lib\subprocess.py", line 1039, in __init__
    self._execute_child(args, executable, preexec_fn, close

KeyboardInterrupt: 

In [9]:
from pathlib import Path
import os
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.decomposition import IncrementalPCA
from threadpoolctl import threadpool_limits

# ŚCIEŻKI
base = Path("D:/ml/undersampling_data/data/unsw/reduced")
base2 = Path("D:/ml/undersampling_data/models/kmeans&centroids2")

df_ = {}

# ⚙️ PARAMETRY
N_THREADS = 16            # lub 24 jeśli CPU i RAM dają radę
pca_components = 50       # liczba wymiarów po redukcji
batch_size = 8192         # rozmiar chunku do PCA
max_iter = 100
n_init = 3
random_state = 42

KM = KMeans(
    n_clusters=int(count2),
    algorithm="elkan",      # szybszy przy Euklidesie
    n_init=n_init,
    max_iter=max_iter,
    random_state=random_state
)

for name in (data.keys() & compare.keys()):
    print(f"\n=== Przetwarzanie: {name} ===")

    # --- przygotowanie danych ---
    X_train, y_train = data[name]
    mask = (y_train == 1)
    X_majority = np.ascontiguousarray(X_train.loc[mask].to_numpy(np.float32, copy=False))

    # --- PCA: dopasowanie i transformacja strumieniowo ---
    print("• Fitting IncrementalPCA...")
    ipca = IncrementalPCA(n_components=pca_components, batch_size=batch_size)
    for start in range(0, X_majority.shape[0], batch_size):
        ipca.partial_fit(X_majority[start:start + batch_size])

    print("• Transforming data with PCA...")
    X_reduced = np.empty((X_majority.shape[0], pca_components), dtype=np.float32)
    for start in range(0, X_majority.shape[0], batch_size):
        X_reduced[start:start + batch_size] = ipca.transform(
            X_majority[start:start + batch_size]
        ).astype(np.float32, copy=False)

    # --- KMeans w zredukowanej przestrzeni ---
    print("• Running KMeans clustering...")
    with threadpool_limits(limits=N_THREADS):
        kmeans = KM.fit(X_reduced)

    # --- Odwrotna transformacja centroidów do oryginalnej przestrzeni ---
    print("• Inverse-transforming centroids...")
    centers = ipca.inverse_transform(kmeans.cluster_centers_).astype(np.float32, copy=False)

    # --- Zbudowanie DataFrame wynikowego ---
    df_majority = pd.DataFrame(centers, columns=X_train.columns, copy=False)
    df_majority["Label"] = 1
    df_majority["source"] = "centroid"

    # dopasowanie kolumn
    df_majority = df_majority.reindex(columns=original_df.columns, fill_value=0.0)
    out = pd.concat([df_majority, original_df], axis=0, ignore_index=True)
    df_[name] = out

    # --- Zapis do pliku ---
    file_path = base / f"{name}_KM_centroids.csv"
    if file_path.exists():
        print(f"File already exists: {file_path.name}")
    else:
        out.to_csv(file_path, index=False)
        print(f"Saved: {file_path.name}")

print("\n✅ Zakończono wszystkie zbiory.")



=== Przetwarzanie: smote ===
• Fitting IncrementalPCA...
• Transforming data with PCA...
• Running KMeans clustering...


KeyboardInterrupt: 

In [ ]:
from threadpoolctl import threadpool_info
print(threadpool_info())


In [8]:
import time, numpy as np
from sklearn.cluster import KMeans
from threadpoolctl import threadpool_limits

# X_full: Twoja macierz (np. float32, C-contiguous)
X = np.ascontiguousarray(X_majority.to_numpy(np.float32, copy=False))
n = X.shape[0]
n_s = max(50_000, n//100)   # ~1%

idx = np.random.RandomState(0).choice(n, n_s, replace=False)
X_s = X[idx]

k = int(13096)
with threadpool_limits(limits=16):             # dostosuj do swojego ustawienia
    t0 = time.perf_counter()
    KMeans(n_clusters=k, algorithm="elkan", n_init="auto", max_iter=100, random_state=42).fit(X_s)
    t1 = time.perf_counter()

t_est = (t1 - t0) * (n / n_s)                  # skalowanie po n
print(f"1%: {t1-t0:.1f}s  → est. full: {t_est/60:.1f} min (dla tych samych wątków)")


1%: 252.7s  → est. full: 330.7 min (dla tych samych wątków)


### KMeans + NN

In [ ]:
base = Path("D:/ml/undersampling_data/data/unsw/reduced")
base2 = Path("D:/ml/undersampling_data/models/kmeans&nn")

cluster_data_ = {
    "mix": {},
    "smote": {},
    "GAN": {},
    "borderline": {}
}
centroids_ = {}

results_ = {}
df_ = {}

results_KM_NN_ = {
    "mix": {},
    "smote": {},
    "GAN": {},
    "borderline": {}
}


for name in data.keys() & compare.keys():
    # Read data from dictionary
    X_train, y_train = data[name]
    compare_df = compare[name]
    
    # Select majority class (synthetic samples)
    X_majority = X_train[y_train == 1]
    
    # Apply KMeans clustering
    KM = KMeans(n_clusters=(int)(count2))
    kmeans = KM.fit(X_majority)
    
    # Save KMeans model
    file_path2 = base2 / f"{name}_kmeans_model.pkl"
    with open(file_path2, "wb") as f:
        pickle.dump(kmeans, f)
    
    # Centroids + Add rows to dictionary
    for i in range ((int)(count2)):
        rows_in_cluster = X_majority[kmeans.labels_ == i] 
        cluster_data_[name][i] = rows_in_cluster
        
        centroids_ = pd.DataFrame(kmeans.cluster_centers_, columns=X_train.columns)
    
    # Original minority class
    for i in range(len(centroids_)):
        if (len(cluster_data_[name][i])>1):     # If more than one sample in the cluster
            dist_={}
            index_ = {}
            centroid = centroids_.iloc[i]
            for j in range(len(cluster_data_[name][i])):
                index_ = list(cluster_data_[name][i].index)
                row = cluster_data_[name][i].iloc[j]
                index_map = {j: idx for j, idx in enumerate(index_)}
                dist_[j] = euclidean(centroid, row)         # Colculate Euclidean distance
                
            min_key = min(dist_, key=dist_.get)
            results_KM_NN_[name][i] = cluster_data_[name][i].iloc[[min_key]]    # Select the nearest neighbor to the centroid
            
        else:           # If only one sample in the cluster
            results_KM_NN_[name][i] = cluster_data_[name][i].iloc[[0]]
        
        results_[name] = pd.concat(results_KM_NN_[name].values(), ignore_index=True)    
    
    # Combine reduced majority class with original df
    df_y_majority = pd.Series([1] * (int)(count2), name="Label") 
    df_majority = pd.concat([results_[name], df_y_majority], axis=1).reset_index(drop=True)
    columns_ = list(df_majority.columns.values)
    df_majority = df_majority.merge(sum_all_data, on=columns_, how="left")
     
    df_[name] = pd.concat([df_majority, original_df], axis=0).reset_index(drop=True) 
    
    print(df_[name].info())
    
    # Save to CSV if file does not exist
    file_path = base / f"{name}_KM_nn.csv"

    if os.path.exists(file_path):
        print("File exists.")
    else:
        df_[name].to_csv(file_path, index=False)
        print("File saved.")

### KMeans + Cosinus Similarity

In [ ]:
base = Path("D:/ml/undersampling_data/data/unsw/reduced")
base2 = Path("D:/ml/undersampling_data/models/kmeans&cos")

cluster_data_ = {
    "mix": {},
    "smote": {},
    "GAN": {},
    "borderline": {}
}
centroids_ = {}

results_ = {}
df_ = {}

results_KM_COS_ = {
    "mix": {},
    "smote": {},
    "GAN": {},
    "borderline": {}
}


for name in data.keys() & compare.keys():
    # Read data from dictionary
    X_train, y_train = data[name]
    compare_df = compare[name]
    
    # Select majority class (synthetic samples)
    X_majority = X_train[y_train == 1]
    
    # Apply KMeans clustering
    KM = KMeans(n_clusters=(int)(count2))
    kmeans = KM.fit(X_majority)
    
    # Save KMeans model
    file_path2 = base2 / f"{name}_kmeans_model.pkl"
    with open(file_path2, "wb") as f:
        pickle.dump(kmeans, f)
    
    # Centroids + Add rows to dictionary
    for i in range ((int)(count2)):
        rows_in_cluster = X_majority[kmeans.labels_ == i] 
        cluster_data_[name][i] = rows_in_cluster
        
        centroids_ = pd.DataFrame(kmeans.cluster_centers_, columns=X_train.columns)
    
    # Original majority class
    for i in range(len(centroids_)):                
        if (len(cluster_data_[name][i])>1):       # If more than one sample in the cluster
            dist_ = {}
            index_ = {}
            centroid = centroids_.iloc[i]
            centroid = centroid.values.reshape(1,-1)
            for j in range(len(cluster_data_[name][i])):
                index_ = list(cluster_data_[name][i].index)
                row = cluster_data_[name][i].iloc[j]
                row = row.values.reshape(1,-1)
                index_map = {j: idx for j, idx in enumerate(index_)}
                dist_[j] = cosine_similarity(centroid, row)         # Colculate Cosine similarity
                
            min_key = max(dist_, key=dist_.get)
            results_KM_COS_[name][i] = cluster_data_[name][i].iloc[[min_key]]   # Select the nearest neighbor to the centroid
            
        else:       # If only one sample in the cluster
            results_KM_COS_[name][i] = cluster_data_[name][i].iloc[[0]]
        
        results_[name] = pd.concat(results_KM_COS_[name].values(), ignore_index=True)       
    
    # Combine reduced majority class with original df
    df_y_majority = pd.Series([1] * (int)(count2), name="target") 
    df_majority = pd.concat([results_[name], df_y_majority], axis=1).reset_index(drop=True)
    columns_ = list(df_majority.columns.values)
    df_majority = df_majority.merge(sum_all_data, on=columns_, how="left")
     
    df_[name] = pd.concat([df_majority, original_df], axis=0).reset_index(drop=True) 
    
    print(df_[name].info())
    
    # Save to CSV if file does not exist
    file_path = base / f"{name}_KM_cos.csv"

    if os.path.exists(file_path):
        print("File exists.")
    else:
        df_[name].to_csv(file_path, index=False)
        print("File saved.")

### KMeans + cos + Mahalanobis distance

In [ ]:
base = Path("D:/ml/undersampling_data/data/unsw/reduced")
base2 = Path("D:/ml/undersampling_data/models/kmeans&cos&mal")

cluster_data_ = {
    "mix": {},
    "smote": {},
    "GAN": {},
    "borderline": {}
}
centroids_ = {}

results_ = {}
df_ = {}

results_KM_COS_MAN_ = {
    "mix": {},
    "smote": {},
    "GAN": {},
    "borderline": {}
}

alfa = 0.5  #wazenie

for name in data.keys() & compare.keys():
    # Read data from dictionary
    X_train, y_train = data[name]
    compare_df = compare[name]
    
    # Select majority class (synthetic samples)
    X_majority = X_train[y_train == 1]
        
    # Apply KMeans clustering
    KM = KMeans(n_clusters=(int)(count2))
    kmeans = KM.fit(X_majority)
    
    # Save KMeans model
    file_path2 = base2 / f"{name}_kmeans_model.pkl"
    with open(file_path2, "wb") as f:
        pickle.dump(kmeans, f)
    
    # Centroids + Add rows to dictionary
    for i in range ((int)(count2)):
        rows_in_cluster = X_majority[kmeans.labels_ == i] 
        cluster_data_[name][i] = rows_in_cluster
        
        centroids_ = pd.DataFrame(kmeans.cluster_centers_, columns=X_train.columns)
    
    # Original majority class
    for i in range(len(centroids_)):               
        if (len(cluster_data_[name][i])>1):       # If more than one sample in the cluster
            cos_={}
            dist_={}
            comb_score_ = {} # results for cosine similarity and Mahalanobis distance
            index_ = {}
            centroid = centroids_.iloc[i]
            centroid = centroid.values.reshape(1,-1)
            for j in range(len(cluster_data_[name][i])):
                index_ = list(cluster_data_[name][i].index)
                row = cluster_data_[name][i].iloc[j]
                row = row.values.reshape(1,-1)
                index_map = {j: idx for j, idx in enumerate(index_)}
                cos_[j] = cosine_similarity(centroid, row)[0,0]          # using Cosine similarity
                dist_[j] = distance.mahalanobis(centroid.flatten(), row.flatten(), np.linalg.pinv(np.cov(X_train.T)))  # using Mahalanobis distance
                
                comb_score_[j] = (alfa*cos_[j] + (1-alfa)*dist_[j])
                
            min_key = min(comb_score_, key=comb_score_.get)
            results_KM_COS_[name][i] = cluster_data_[name][i].iloc[[min_key]]
            
        else:
            results_KM_COS_[name][i] = cluster_data_[name][i].iloc[[0]]
        
        results_[name] = pd.concat(results_KM_COS_[name].values(), ignore_index=True)   
    
    # Combine reduced majority class with original df
    df_y_majority = pd.Series([1] * (int)(count2), name="target") 
    df_majority = pd.concat([results_[name], df_y_majority], axis=1).reset_index(drop=True)
    columns_ = list(df_majority.columns.values)
    df_majority = df_majority.merge(sum_all_data, on=columns_, how="left")
     
    df_[name] = pd.concat([df_majority, original_df], axis=0).reset_index(drop=True) 
    
    print(df_[name].info())
    
    # Save to CSV if file does not exist
    file_path = base / f"{name}_KM_cos_mal.csv"

    if os.path.exists(file_path):
        print("File exists.")
    else:
        df_[name].to_csv(file_path, index=False)
        print("File saved.")

### HDBSCAN + Euclidean distance

In [ ]:
base = Path("D:/ml/undersampling_data/data/unsw/reduced")
base2 = Path("D:/ml/undersampling_data/models/hdbscan&nn")

cluster_data_ = {
    "mix": {},
    "smote": {},
    "GAN": {},
    "borderline": {}
}
centroids_ = {}

results_ = {}
df_ = {}

results_HDBSCAN_DIST_ = {
    "mix": {},
    "smote": {},
    "GAN": {},
    "borderline": {}
}
labels_={}

for name in data.keys() & compare.keys():
    # Read data from dictionary
    X_train, y_train = data[name]
    compare_df = compare[name]
    
    # Select majority class (synthetic samples)
    X_majority = X_train[y_train == 1]
    
    # Apply HDBSCAN clustering
    hdbscan = HDBSCAN(store_centers="centroid")
    hdbscan_res = hdbscan.fit(X_majority)
    
    # Save HDBSCAN model
    file_path2 = base2 / f"{name}_hdbscan_model.pkl"
    with open(file_path2, "wb") as f:
        pickle.dump(hdbscan, f)
    
    # Get labels and number of clusters
    labels = hdbscan_res.labels_
    unique_lables = np.unique(labels)
    labels_[name] = len(unique_lables[unique_lables >=0])
    labels = len(unique_lables[unique_lables >=0])
    print(f"{name}: {labels_[name]}")
    
    # Centroids from HDBSCAN
    centroids_ = pd.DataFrame(hdbscan_res.centroids_, columns=X_train.columns)
    centroids_hdbscan = hdbscan.fit_predict(X_majority)

    # Add rows to cluster dictionary
    for i in range(labels):
        rows_in_cluster = X_majority[hdbscan.labels_ == i] 
        cluster_data_[name][i] = rows_in_cluster

        target = count2
        
        # Sorted neighbors dictionary
        per_cluster_sorted = {}
        
        # Calculate the nieghbors for each centroid (centroid -> rows_in_cluster)
    for i in range(len(centroids_)):
        rows = cluster_data_[name][i]
        if len(rows) == 0:
            per_cluster_sorted[i] = []
            continue
        if len(rows) == 1:
            per_cluster_sorted[i] = [rows.index[0]]
            continue

        centroid = centroids_.iloc[i].to_numpy()
        # Calculate Euclidean distances from centroid to all points in the cluster
        dists = rows.apply(lambda r: euclidean(centroid, r.to_numpy()), axis=1)
        order = dists.sort_values().index.tolist()  # indeksy X_majority w kolejności rosnących dystansów
        per_cluster_sorted[i] = order

    # Round-robin chooice to target_n
    selected_idx = []
    ptr = {i: 0 for i in range(len(centroids_))}                     # Which "index" to take now from the cluster
    while len(selected_idx) < target:
        progressed = False
        for i in range(len(centroids_)):
            lst = per_cluster_sorted[i]
            j = ptr[i]
            if j < len(lst):
                idx = lst[j]
                if idx not in selected_idx:
                    selected_idx.append(idx)
                ptr[i] += 1
                progressed = True
                if len(selected_idx) >= target:
                    break
        if not progressed:
            break

    # Choose selected records of majority class (in order of selection)
    maj_selected = X_majority.loc[selected_idx].reset_index(drop=True)
    results_[name] = maj_selected   
    
    # Combine reduced majority class with original df    
    df_y_majority = pd.Series([1.0] * len(results_[name]), name="target")
    df_majority = pd.concat([results_[name], df_y_majority], axis=1).reset_index(drop=True)
    columns_ = list(df_majority.columns.values)
    df_majority = df_majority.merge(sum_all_data, on=columns_, how="left")
     
    df_[name] = pd.concat([df_majority, original_df], axis=0).reset_index(drop=True)  
    #print(df_[name])

    print(df_[name].info())

    # Save to CSV if file does not exist
    file_path = base / f"{name}_HDBSCAN_NN.csv"

    if os.path.exists(file_path):
        print("File exists.")
    else:
        df_[name].to_csv(file_path, index=False)
        print("File saved.")

### HDBSCAN + Cosinus Similarity

In [ ]:
base = Path("D:/ml/undersampling_data/data/unsw/reduced")
base2 = Path("D:/ml/undersampling_data/models/hdbscan&cos")

cluster_data_ = {
    "mix": {},
    "smote": {},
    "GAN": {},
    "borderline": {}
}
centroids_ = {}

results_ = {}
df_ = {}

results_HDBSCAN_COS_ = {
    "mix": {},
    "smote": {},
    "GAN": {},
    "borderline": {}
}


for name in data.keys() & compare.keys():
    # Read data from dictionary
    X_train, y_train = data[name]
    compare_df = compare[name]
    
    # Select majority class (synthetic samples)
    X_majority = X_train[y_train == 1]
    
    # Apply HDBSCAN clustering
    hdbscan = HDBSCAN(store_centers="centroid")
    hdbscan_res = hdbscan.fit(X_majority)
    
    # Save HDBSCAN model
    file_path2 = base2 / f"{name}_hdbscan_model.pkl"
    with open(file_path2, "wb") as f:
        pickle.dump(hdbscan, f)
        
    # Get labels and number of clusters
    labels = hdbscan_res.labels_
    unique_lables = np.unique(labels)
    labels = len(unique_lables[unique_lables >=0])
    centroids_ = pd.DataFrame(hdbscan_res.centroids_, columns=X_train.columns)
    
    centroids_hdbscan = hdbscan.fit_predict(X_majority)

    # Centroids from HDBSCAN + Add rows to cluster dictionary
    for i in range(labels):
        # Add rows to cluster dictionary
        rows_in_cluster = X_majority[hdbscan.labels_ == i] 
        cluster_data_[name][i] = rows_in_cluster
        
        target = count2

        # Sorted crucial indexes from clusters
        per_cluster_sorted = {}
        
        # Calculate the nieghbors for each centroid (centroid -> rows_in_cluster)
    for i in range(len(centroids_)):
        rows = cluster_data_[name][i]
        if len(rows) == 0:
            per_cluster_sorted[i] = []
            continue
        if len(rows) == 1:
            per_cluster_sorted[i] = [rows.index[0]]
            continue

        centroid = centroids_.iloc[i].to_numpy().reshape(1, -1)
        # Calculate Cosine distances from centroid to all points in the cluster
        dists = pairwise_distances(rows.values, centroid, metric="cosine").ravel()
        order = rows.index[np.argsort(dists)].tolist()  # indeksy X_majority w kolejności rosnących dystansów
        per_cluster_sorted[i] = order

    # Round-robin chooice to target_n
    selected_idx = []
    ptr = {i: 0 for i in range(len(centroids_))}                 # which "index" to take now from the cluster
    while len(selected_idx) < target:
        progressed = False
        for i in range(len(centroids_)):
            lst = per_cluster_sorted[i]
            j = ptr[i]
            if j < len(lst):
                idx = lst[j]
                if idx not in selected_idx:
                    selected_idx.append(idx)
                ptr[i] += 1
                progressed = True
                if len(selected_idx) >= target:
                    break
        if not progressed: 
            break

    # Choose selected records of majority class (in order of selection)
    maj_selected = X_majority.loc[selected_idx].reset_index(drop=True)
    results_[name] = maj_selected   
    
    # Combine reduced majority class with original df
    df_y_majority = pd.Series([1.0] * len(results_[name]), name="target")
    df_majority = pd.concat([results_[name], df_y_majority], axis=1).reset_index(drop=True)
    columns_ = list(df_majority.columns.values)
    df_majority = df_majority.merge(sum_all_data, on=columns_, how="left")
        
    df_[name] = pd.concat([df_majority, original_df], axis=0).reset_index(drop=True)
    
    print(df_[name].info())
    
    # Save to CSV if file does not exist
    file_path = base / f"{name}_HDBSCAN_cos.csv"

    if os.path.exists(file_path):
        print("File exists.")
    else:
        df_[name].to_csv(file_path, index=False)
        print("File saved.")

### KMeans (samples=HDBSCAN()) + NN

In [ ]:
base = Path("D:/ml/undersampling_data/data/unsw/reduced")
base2 = Path("D:/ml/undersampling_data/models/kmeans&hdbscan&nn")

cluster_data_ = {
    "mix": {},
    "smote": {},
    "GAN": {},
    "borderline": {}
}
centroids_ = {}

results_ = {}
df_ = {}

results_KMEANS_HDBSCAN_DIST_ = {
    "mix": {},
    "smote": {},
    "GAN": {},
    "borderline": {}
}

for name in data.keys() & compare.keys():
    # Read data from dictionary
    X_train, y_train = data[name]
    compare_df = compare[name]
    
    # Select majority class (synthetic samples)
    X_majority = X_train[y_train == 1]
    
    # Apply KMeans clustering
    KM = KMeans(n_clusters=labels_[name])
    kmeans = KM.fit(X_majority)
    
    # Save KMeans model
    file_path2 = base2 / f"{name}_kmeans_model.pkl"
    with open(file_path2, "wb") as f:
        pickle.dump(kmeans, f)
        
    # Centroids from KMeans + Add rows to cluster dictionary
    for i in range (labels_[name]):
        # Add rows to cluster dictionary
        rows_in_cluster = X_majority[kmeans.labels_ == i] 
        cluster_data_[name][i] = rows_in_cluster
        
        centroids_ = pd.DataFrame(kmeans.cluster_centers_, columns=X_train.columns)
    
    target = count2
    
    # Sorted crucial indexes from clusters
    per_cluster_sorted = {}
    for i in range(len(centroids_)):
        rows = cluster_data_[name][i]
        if len(rows) == 0:
            per_cluster_sorted[i] = []
            continue
        if len(rows) == 1:
            per_cluster_sorted[i] = [rows.index[0]]
            continue
             
        centroid = centroids_.iloc[i].to_numpy()
        # Calculate Euclidean distances from centroid to all points in the cluster
        dists = rows.apply(lambda r: euclidean(centroid, r.to_numpy()), axis=1)
        order = dists.sort_values().index.tolist()  # indexes of X_majority in ascending order of distances
        per_cluster_sorted[i] = order
        
    selected_idx = []
    ptr = {i: 0 for i in range(len(centroids_))}                     # which "index" to take now from the cluster
    while len(selected_idx) < target:
        progressed = False
        for i in range(len(centroids_)):
            lst = per_cluster_sorted[i]
            j = ptr[i]
            if j < len(lst):
                idx = lst[j]
                if idx not in selected_idx:
                    selected_idx.append(idx)
                ptr[i] += 1
                progressed = True
                if len(selected_idx) >= target:
                    break
        if not progressed: 
            break
    
    # Choose selected records of majority class (in order of selection)   
    maj_selected = X_majority.loc[selected_idx].reset_index(drop=True)   
    results_[name] = maj_selected     
    
    # Combine reduced majority class with original df
    df_y_majority = pd.Series([1.0] * len(results_[name]), name="target")
    df_majority = pd.concat([results_[name], df_y_majority], axis=1).reset_index(drop=True)
    columns_ = list(df_majority.columns.values)
    df_majority = df_majority.merge(sum_all_data, on=columns_, how="left")
        
    df_[name] = pd.concat([df_majority, original_df], axis=0).reset_index(drop=True) 
    
    print(df_[name].info()) 
    
    # Save to CSV if file does not exist
    file_path = base / f"{name}_kmeans&hdbscan_nn.csv"

    if os.path.exists(file_path):
        print("File exists.")
    else:
        df_[name].to_csv(file_path, index=False)
        print("File saved.")

### KMeans (samples=HDBSCAN()) + cosinus

In [ ]:
base = Path("D:/ml/undersampling_data/data/unsw/reduced")
base2 = Path("D:/ml/undersampling_data/models/kmeans&hdbscan&cos")

cluster_data_ = {
    "mix": {},
    "smote": {},
    "GAN": {},
    "borderline": {}
}
centroids_ = {}

results_ = {}
df_ = {}

results_KMEANS_HDBSCAN_DIST_ = {
    "mix": {},
    "smote": {},
    "GAN": {},
    "borderline": {}
}

for name in data.keys() & compare.keys():
    # Read data from dictionary
    X_train, y_train = data[name]
    compare_df = compare[name]
    
    # Select majority class (synthetic samples) 
    X_majority = X_train[y_train == 1]
    
    # Apply KMeans clustering
    KM = KMeans(n_clusters=labels_[name])
    kmeans = KM.fit(X_majority)
    
    # Save KMeans model
    file_path2 = base2 / f"{name}_kmeans_model.pkl"
    with open(file_path2, "wb") as f:
        pickle.dump(kmeans, f)
    
    # Centroids from KMeans + Add rows to cluster dictionary
    for i in range (labels_[name]):
        rows_in_cluster = X_majority[kmeans.labels_ == i] 
        cluster_data_[name][i] = rows_in_cluster
        
        centroids_ = pd.DataFrame(kmeans.cluster_centers_, columns=X_train.columns)
    
    target = count2
    
    # Sorted crucial indexes from clusters
    per_cluster_sorted = {}
    for i in range(len(centroids_)):
        rows = cluster_data_[name][i]
        if len(rows) == 0:
            per_cluster_sorted[i] = []
            continue
        if len(rows) == 1:
            per_cluster_sorted[i] = [rows.index[0]]
            continue
             
        centroid = centroids_.iloc[i].to_numpy().reshape(1, -1)
        # Calculate Cosine distances from centroid to all points in the cluster
        dists = pairwise_distances(rows.values, centroid, metric="cosine").ravel()
        order = rows.index[np.argsort(dists)].tolist()              # indexes of X_majority in ascending order of distances
        per_cluster_sorted[i] = order
        
    selected_idx = []
    ptr = {i: 0 for i in range(len(centroids_))}                     # which "index" to take now from the cluster
    while len(selected_idx) < target:
        progressed = False
        for i in range(len(centroids_)):
            lst = per_cluster_sorted[i]
            j = ptr[i]
            if j < len(lst):
                idx = lst[j]
                if idx not in selected_idx:
                    selected_idx.append(idx)
                ptr[i] += 1
                progressed = True
                if len(selected_idx) >= target:
                    break
        if not progressed:  # wszystkie klastry wyczerpane
            break
        
    # Select selected records of majority class (in order of selection)    
    maj_selected = X_majority.loc[selected_idx].reset_index(drop=True)   # <-- CHANGED
    results_[name] = maj_selected     
    
    # Combine reduced majority class with original df
    df_y_majority = pd.Series([1.0] * len(results_[name]), name="target")
    df_majority = pd.concat([results_[name], df_y_majority], axis=1).reset_index(drop=True)
    columns_ = list(df_majority.columns.values)
    df_majority = df_majority.merge(sum_all_data, on=columns_, how="left")
    
    df_[name] = pd.concat([df_majority, original_df], axis=0).reset_index(drop=True) 
    print(df_[name].info()) 
    
    # Save to CSV if file does not exist
    file_path = base / f"{name}_kmeans&hdbscan_cos.csv"

    if os.path.exists(file_path):
        print("File exists.")
    else:
        df_[name].to_csv(file_path, index=False)
        print("File saved.")

### KMeans (samples=HDBSCAN()) + cosinus + Mahalanobis

In [ ]:
base = Path("D:/ml/undersampling_data/data/unsw/reduced")
base2 = Path("D:/ml/undersampling_data/models/kmeans&hdbscan&cos&mal")

cluster_data_ = {
    "mix": {},
    "smote": {},
    "GAN": {},
    "borderline": {}
}
centroids_ = {}

results_ = {}
df_ = {}

results_KMEANS_HDBSCAN_COS_NN_ = {
    "mix": {},
    "smote": {},
    "GAN": {},
    "borderline": {}
}

alfa = 0.5

for name in data.keys() & compare.keys():
    # Read data from dictionary
    X_train, y_train = data[name]
    compare_df = compare[name]
    
    # Select majority class (synthetic samples)
    X_majority = X_train[y_train == 1]
    
    # Apply KMeans clustering
    KM = KMeans(n_clusters=labels_[name])
    kmeans = KM.fit(X_majority)
    
    # Save KMeans model
    file_path2 = base2 / f"{name}_kmeans_model.pkl"
    with open(file_path2, "wb") as f:
        pickle.dump(kmeans, f)
    
    # Centroids from KMeans + Add rows to cluster dictionary
    for i in range (labels_[name]):
        rows_in_cluster = X_majority[kmeans.labels_ == i] 
        cluster_data_[name][i] = rows_in_cluster
        
        centroids_ = pd.DataFrame(kmeans.cluster_centers_, columns=X_train.columns)
        
    target = count2
    
    # Sorted crucial indexes from clusters
    per_cluster_sorted = {}
    for i in range(len(centroids_)):                # for each centroid
        if (len(cluster_data_[name][i])>1):        # check if more than one sample in the cluster
            cos_={}
            dist_={}
            comb_score_ = {}                # results for cosine similarity and Mahalanobis distance
            index_ = {}
            centroid = centroids_.iloc[i]
            centroid = centroid.values.reshape(1,-1)
            for j in range(len(cluster_data_[name][i])):
                index_ = list(cluster_data_[name][i].index)
                row = cluster_data_[name][i].iloc[j]
                row = row.values.reshape(1,-1)
                index_map = {j: idx for j, idx in enumerate(index_)}
                cos_[j] = cosine_similarity(centroid, row)[0,0]         #using Cosine similarity
                dist_[j] = distance.mahalanobis(centroid.flatten(), row.flatten(), np.linalg.pinv(np.cov(X_train.T)))  #using Mahalanobis distance
                
                comb_score_[j] = (alfa*cos_[j] + (1-alfa)*dist_[j])
                
            min_key = min(comb_score_, key=comb_score_.get)
            results_KMEANS_HDBSCAN_COS_NN_[name][i] = cluster_data_[name][i].iloc[[min_key]]
            
            order = sorted(comb_score_, key=comb_score_.get, reverse=True)
            per_cluster_sorted[i] = [cluster_data_[name][i].index[j] for j in order]
                
        else:
            results_KMEANS_HDBSCAN_COS_NN_[name][i] = cluster_data_[name][i].iloc[[0]]
        
    results_[name] = pd.concat(results_KMEANS_HDBSCAN_COS_NN_[name].values(), ignore_index=True)
    
    # Sorted crucial indexes from clusters
    selected_idx = []
    ptr = {i: 0 for i in range(len(centroids_))}                     # which "index" to take now from the cluster
    while len(selected_idx) < target:
        progressed = False
        for i in range(len(centroids_)):
            lst = per_cluster_sorted.get(i, [])
            j = ptr[i]
            if j < len(lst):
                idx = lst[j]
                if idx not in selected_idx:
                    selected_idx.append(idx)
                ptr[i] += 1
                progressed = True
                if len(selected_idx) >= target:
                    break
        if not progressed:  
            break

    # Choose selected records of majority class (in order of selection)
    maj_selected = X_majority.loc[selected_idx].reset_index(drop=True)
    results_[name] = maj_selected     
    
    # Combine reduced majority class with original df
    df_y_majority = pd.Series([1.0] * len(results_[name]), name="target")
    df_majority = pd.concat([results_[name], df_y_majority], axis=1).reset_index(drop=True)
    columns_ = list(df_majority.columns.values)
    df_majority = df_majority.merge(sum_all_data, on=columns_, how="left")
    
    df_[name] = pd.concat([df_majority, original_df], axis=0).reset_index(drop=True) 
    print(df_[name].info()) 
    
    # Save to CSV if file does not exist
    file_path = base / f"{name}_kmeans&hdbscan_cos&mal.csv"

    if os.path.exists(file_path):
        print("File exists.")
    else:
        df_[name].to_csv(file_path, index=False)
        print("File saved.")